In [32]:
from torch.distributions import Normal

import numpy as np
import torch
import pickle
from tqdm import tqdm
from joblib import Parallel, delayed
from pyro.distributions.empirical import Empirical

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

# sbi
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils
from sbi import analysis

# distances
from scipy.spatial.distance import directed_hausdorff

# Normal distribution

In [5]:
def noisy_simulator(noise_param):
    
    def wrapped(input):

        dist = Normal(input,1)
        real_sample = dist.sample()
        noise_dist = Normal(0,1)
        noise_sample = noise_dist.sample()

        r = np.random.random_sample()

        return noise_sample if r < noise_param else real_sample


    return wrapped


Define prior.

In [16]:
prior = Normal(0,1)
true_val = prior.sample()
noise_param = 0.2
obs = noisy_simulator(noise_param)(true_val)

Simulator wrapper to produce distances.

In [30]:
# def sim_distances_wrapper(noise_param):

#     simulator = noisy_simulator(noise_param)
    
#     def distance_sum(input):
#         return torch.abs(simulator(input)-obs)

#     return distance_sum

def sim_distances_wrapper(input):
    
    return torch.abs(noisy_simulator(noise_param)(input)-obs).reshape(-1)

In [39]:
def true_posterior(observation):
    # Assumes likelihood std = 1, prior std = 1, prior mean = 0.
    posterior_var = torch.tensor(1/2)
    posterior_mu = posterior_var * obs
    return Normal(posterior_mu, torch.sqrt(posterior_var))

def get_dist_pdf(dist):
    return lambda x: torch.exp(dist.log_prob(x))
    

In [42]:
get_dist_pdf(true_posterior(obs))(torch.tensor(1))

tensor(0.4153)